In [47]:
import os
import pandas as pd

# 예제 경로 설정
paths = {
    'TRAIN_DATA_PATH': r"C:\Users\조여원\train_data.csv",
    'VALID_DATA_PATH': r"C:\Users\조여원\valid_data.csv",
    'TEST_DATA_PATH': r"C:\Users\조여원\test_data.csv"
}

# 각 경로를 UTF-8로 인코딩하여 환경 변수에 저장
for key, path in paths.items():
    encoded_path = path.encode('utf-8')
    os.environ[key] = encoded_path.decode('utf-8')

# JOBLIB_TEMP_FOLDER 환경 변수 설정 (예제 경로 중 하나 사용)
os.environ['JOBLIB_TEMP_FOLDER'] = paths['TRAIN_DATA_PATH'].encode('utf-8').decode('utf-8')

# 환경 변수에서 파일 경로 가져오기
train_data_path = os.environ['TRAIN_DATA_PATH']
valid_data_path = os.environ['VALID_DATA_PATH']
test_data_path = os.environ['TEST_DATA_PATH']

# JOBLIB_TEMP_FOLDER 환경 변수 확인
joblib_temp_folder = os.environ['JOBLIB_TEMP_FOLDER']
print("JOBLIB_TEMP_FOLDER:", joblib_temp_folder)

# 경로를 출력하여 확인
print("Train Data Path:", train_data_path)
print("Valid Data Path:", valid_data_path)
print("Test Data Path:", test_data_path)

# 데이터 파일 불러오기
df_train = pd.read_csv(train_data_path)
df_valid = pd.read_csv(valid_data_path)
df_test = pd.read_csv(test_data_path)

# 데이터 확인
print("Train Data:")
print(df_train.head())

print("Validation Data:")
print(df_valid.head())

print("Test Data:")
print(df_test.head())


JOBLIB_TEMP_FOLDER: C:\Users\조여원\train_data.csv
Train Data Path: C:\Users\조여원\train_data.csv
Valid Data Path: C:\Users\조여원\valid_data.csv
Test Data Path: C:\Users\조여원\test_data.csv
Train Data:
           site                 sid     ldate  lrate  llevel leaktype  0HZ  \
0  S-4677025028  S-0359369083963450  20190309     90     562       in    0   
1  S-2911010107  S-0359369084047519  20201122     30     267   normal    0   
2  S-2911010107  S-0359369084050018  20201202     30     201   normal    0   
3  S-4682025024  S-0359369085870067  20201228     90     870    other    0   
4  S-4677025029  S-0359369081311207  20200428     90     183      out    0   

   10HZ  20HZ  30HZ  ...  MAX10  MAX11  MAX12  MAX13  MAX14  MAX15  MAX16  \
0     9     9    13  ...    350    519    350    543    350    602    350   
1     4     4     4  ...    270     83    260     92    260     78    270   
2     4     4     4  ...    370    102    380     98    370    253    370   
3     0     0     0  ...    27

In [48]:


# 사용할 컬럼 설정
columns = ["llevel", "lrate", "90HZ", "100HZ", "110HZ", "120HZ", "130HZ", "140HZ", "150HZ", "160HZ", 
           "170HZ", "180HZ", "190HZ", "200HZ", "210HZ", "220HZ", "230HZ", "240HZ", "250HZ", "260HZ", 
           "270HZ", "280HZ", "290HZ", "300HZ", "310HZ", "320HZ", "330HZ", "340HZ", "350HZ", "360HZ", 
           "370HZ", "380HZ", "390HZ", "400HZ", "410HZ", "420HZ", "430HZ", "440HZ", "450HZ", "460HZ", 
           "470HZ", "480HZ", "510HZ", "540HZ", "560HZ", "720HZ", "730HZ", "740HZ", "760HZ", "770HZ", 
           "780HZ", "830HZ", "840HZ"]

In [49]:
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [50]:
# x_train, y_train 설정
x_train = df_train[columns]
y_train = df_train.iloc[:, 5]

# x_valid, y_valid 설정
x_valid = df_valid[columns]
y_valid = df_valid.iloc[:, 5]

# x_test, y_test 설정
x_test = df_test[columns]
y_test = df_test.iloc[:, 5]

In [51]:
# 라벨 인코딩
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_valid = label_encoder.transform(y_valid)
y_test = label_encoder.transform(y_test)

In [52]:


import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# 고정된 파라미터
params = {
    'objective': 'multi:softprob',
    'num_class': 3  # 클래스 개수에 맞게 설정
}


In [53]:
# 조정할 하이퍼파라미터 그리드 설정
param_grid = {
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.3],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 300, 500],
}

In [54]:
# XGBoost 모델 초기화
xgb_model = xgb.XGBClassifier(**params)

In [55]:
# GridSearchCV 설정
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)


In [56]:
# Grid Search 실행
grid_search.fit(x_train, y_train)

UnicodeEncodeError: 'ascii' codec can't encode characters in position 18-20: ordinal not in range(128)

In [ ]:
# 최적의 하이퍼파라미터 출력
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: {:.2f}".format(grid_search.best_score_))

In [ ]:
# 최적의 모델 추출 및 평가
best_xgb_model = grid_search.best_estimator_
y_pred = best_xgb_model.predict(x_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy of the best model: {:.2f}".format(test_accuracy))